<a href="https://colab.research.google.com/github/AlexanderLontke/ssl-remote-sensing/blob/bigan/BiGAN_allChannels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BiGAN
Implementation with all 13 channels of BigEarthNet-S2

In [1]:
!pip install ssl_remote_sensing@git+https://github.com/AlexanderLontke/ssl-remote-sensing.git@bigan

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/AlexanderLontke/ssl-remote-sensing.git (to revision bigan) to /tmp/pip-install-d6eu_s7u/ssl-remote-sensing_a4d466c0fad046a0a615fbadbe00eddd
  Running command git clone -q https://github.com/AlexanderLontke/ssl-remote-sensing.git /tmp/pip-install-d6eu_s7u/ssl-remote-sensing_a4d466c0fad046a0a615fbadbe00eddd
  Running command git checkout -b bigan --track origin/bigan
  Switched to a new branch 'bigan'
  Branch 'bigan' set up to track remote branch 'bigan' from 'origin'.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for ssl-remote-sensing: filename=ssl_remote_sensing-0.0.post1.dev179+gc02170a-py3-none-any.whl size=44362 sha256=5ee3996b378dcc7d521f919558554ebc3269083198501866a6e1f50e74102257
  Stored in directory: /tmp/pip-ephe

In [2]:
import torch
import os
import glob
from pytorch_lightning import Trainer

In [3]:
from ssl_remote_sensing.pretext_tasks.gan.bigan_generator import BiganResnetGenerator
from ssl_remote_sensing.pretext_tasks.gan.bigan_encoder import BiganResnetEncoder
from ssl_remote_sensing.pretext_tasks.gan.bigan_discriminator import BiganDiscriminator
from ssl_remote_sensing.pretext_tasks.gan.bigan import BIGAN
from ssl_remote_sensing.pretext_tasks.gan.config import get_bigan_config
from ssl_remote_sensing.pretext_tasks.gan.utils import reproducibility
from ssl_remote_sensing.data.bigearthnet_dataloader import BigearthnetDataModule

In [4]:

# Log in to your W&B account
import wandb

wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: marccgrau (unisg-ds-nlp). Use `wandb login --relogin` to force relogin


True

In [5]:
# import the Google Colab GDrive connector
from google.colab import drive

# mount GDrive inside the Colab notebook
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Machine setup
available_gpus = torch.cuda.device_count()
save_model_path = os.path.join(os.getcwd(), "saved_models/")
print("available_gpus:", available_gpus)

available_gpus: 1


In [7]:
# Model Setup
train_config = get_bigan_config()

In [8]:
train_config.cuda = True
train_config.batch_size = 2000

In [9]:
#!cp -r /content/drive/MyDrive/bigearthnet.tar.gz /content/data/

In [10]:
#!tar -xf /content/data/bigearthnet.tar.gz 

In [11]:
dataset_dir = "/content/BigEarthNet-v1.0/"

In [12]:
datamodule = BigearthnetDataModule(data_dir = dataset_dir, num_workers = 2)

In [13]:
datamodule.setup()

In [14]:
dataloader = datamodule.train_dataloader()

In [15]:
# Number of channels in the training images. For color images this is 3
nc = 12

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

In [16]:
model = BIGAN(image_channels=12)

In [17]:
trainer = Trainer(max_epochs = 10, accelerator='gpu', devices=1)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, dataloader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.8/dist-packages/torch/optim/adam.py:90: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  super(Adam, self).__init__(params, defaults)
INFO:pytorch_lightning.callbacks.model_summary:
  | Name             | Type                 | Params
----------------------------------------------------------
0 | pretrained_model | ResNet               | 11.2 M
1 | generator        | BiganResnetGenerator | 21.8 M
2 | discriminator    | BiganDiscriminator   | 9.4 M 
3 | encoder          | BiganResnetEncoder   | 11.3 M
4 | criterion        | BCELoss              | 0     
----------------------------------------------------------
31.2 M    Trainable params
0         Non-trainable params
31.2 M    Total params
124.702   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

In [ ]:
print(model)